### GRU总览

GRU 于普通的 RNN 的区别在于对于潜变量的更新

GRU 在对潜变量进行更新时会考虑对历史序列信息的保留程度（这个部分也需要设计为模型参数进行学习）

引入了更新门、重置门和候选潜变量的概念，这三个可以理解为用于计算最终更新的潜变量的中间值

更新门（Update Gate）：
$$
z_t = \sigma(W_z x_t + U_z h_{t-1} + b_z)
$$

重置门（Reset Gate）：
$$
r_t = \sigma(W_r x_t + U_r h_{t-1} + b_r)
$$

候选隐藏状态（Candidate Hidden State）：
$$
\tilde{h}_t = \tanh(W_h x_t + U_h (r_t \odot h_{t-1}) + b_h)
$$

最终隐藏状态（New Hidden State）：
$$
h_t = (1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t
$$


使用 GRU 更新一次潜变量的图解如下：

![](md-img/GRU.jpg)

其中箭头表示全连接

对于更新门和重置门的激活函数使用 Sigmoid 函数

对于候选潜变量的激活函数使用 tanh 函数

使用最终得到的潜变量即可进行下一个 token 的预测（同 RNN）

$$
其中 H、Z、R、\tilde{H} 形状一致
$$